# Data Cleaning

## Introduction

This notebook goes through a necessary step of any data science project - data cleaning. Data cleaning is a time consuming and unenjoyable task, yet it's a very important one. Keep in mind, "garbage in, garbage out". Feeding dirty data into a model will give us results that are meaningless.

Specifically, we'll be walking through:

1. **Getting the data - **in this case, we'll be scraping data from a website
2. **Cleaning the data - **we will walk through popular text pre-processing techniques
3. **Organizing the data - **we will organize the cleaned data into a way that is easy to input into other algorithms

The output of this notebook will be clean, organized data in two standard text formats:

1. **Corpus** - a collection of text
2. **Document-Term Matrix** - word counts in matrix format

## Problem Statement

As a reminder, our goal is to look at transcripts of various comedians and note their similarities and differences. Specifically, I'd like to know if Ali Wong's comedy style is different than other comedians, since she's the comedian that got me interested in stand up comedy.

## Getting The Data

Luckily, there are wonderful people online that keep track of stand up routine transcripts. [Scraps From The Loft](http://scrapsfromtheloft.com) makes them available for non-profit and educational purposes.

To decide which comedians to look into, I went on IMDB and looked specifically at comedy specials that were released in the past 5 years. To narrow it down further, I looked only at those with greater than a 7.5/10 rating and more than 2000 votes. If a comedian had multiple specials that fit those requirements, I would pick the most highly rated one. I ended up with a dozen comedy specials.

In [3]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# get and parse main page to get collection of links
main_url = "https://www.vie-publique.fr/discours-dans-lactualite/269998-les-voeux-des-presidents-de-la-republique-depuis-1974"
page = requests.get(main_url).text
soup = BeautifulSoup(page, "lxml")
links = soup.find(class_="container88").find_all('a')


# from links collection, get all pairs of name+year and url
dates=[]
urls=[]
for link in links:
    title = link.text.split(",")
    url = link['href']
    # print(title, url)
    if len(title)==2 and (url.startswith('https://www.vie-publique.fr') or url.startswith('/discours')):
        date = title[0]+" "+title[1].split()[-1]
        dates.append(date)
        url = link['href']
        if not url.startswith('https://www.vie-publique.fr'):
            url = 'https://www.vie-publique.fr' + url
        urls.append(url)
    else:
        pass
        # print('rejected')
print('found ',len(dates), ' discours')

# get text from all subpage
transcripts=[]
for url in urls:
    # print(url)
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = soup.find(class_="field--name-field-texte-integral").text
    transcripts.append(text)
    # print(len(text))

print([len(text) for text in transcripts])

43  discours
[12123, 14291, 13482, 13270, 7937, 8018, 8480, 6711, 7213, 6777, 6664, 8016, 6390, 5577, 7478, 6688, 6009, 7872, 7060, 6225, 5398, 7228, 8243, 6014, 5291, 5395, 5011, 5011, 4950, 5045, 3695, 3815, 3387, 2926, 2636, 5723, 6698, 5455, 6425, 3688, 4216, 4542, 7234]


In [ ]:
# # Actually request transcripts (takes a few minutes to run)
# transcripts = [url_to_transcript(u) for u in urls]

In [4]:
# # Pickle files for later use

# # Make a new directory to hold the text files
# !mkdir transcripts

for i, c in enumerate(dates):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)
pickle.dump( data, open( "data_fr.pkl", "wb" ) )

NameError: name 'data' is not defined

In [ ]:
# Load pickled files
data = {}
for i, c in enumerate(dates):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [53]:
# Double check to make sure data has been loaded properly
data.keys()

dict_keys(['Emmanuel Macron 2020', 'Emmanuel Macron 2019', 'Emmanuel Macron 2018', 'Emmanuel Macron 2017', 'François Hollande 2016', 'François Hollande 2015', 'François Hollande 2014', 'François Hollande 2012', 'Nicolas Sarkozy 2011', 'Nicolas Sarkozy 2010', 'Nicolas Sarkozy 2009', 'Nicolas Sarkozy 2007', 'Jacques Chirac 2006', 'Jacques Chirac 2005', 'Jacques Chirac 2004', 'Jacques Chirac 2003', 'Jacques Chirac 2002', 'Jacques Chirac 2001', 'Jacques Chirac 1999', 'Jacques Chirac 1998', 'Jacques Chirac 1997', 'Jacques Chirac 1996', 'Jacques Chirac 1995', 'François Mitterrand 1994', 'François Mitterrand 1993', 'François Mitterrand 1992', 'François Mitterrand 1991', 'François Mitterrand 1990', 'François Mitterrand 1989', 'François Mitterrand 1988', 'François Mitterrand 1987', 'François Mitterrand 1986', 'François Mitterrand 1985', 'François Mitterrand 1984', 'François Mitterrand 1983', 'François Mitterrand 1982', 'François Mitterrand 1981', 'Valéry Giscard d’Estaing 1980', 'Valéry Giscard

In [ ]:
# More checks
data['Nicolas Sarkozy 2010'][10:20]

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

In [ ]:
# Let's take a look at our data again
next(iter(data.keys()))

In [ ]:
# Notice that our dictionary is currently in key: comedian, value: list of text format
next(iter(data.values()))

In [ ]:
# We are going to change this to key: comedian, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
# Combine it!
#data_combined = {key: [combine_text(value)] for (key, value) in data.items()}
data_combined = {key: [value] for (key, value) in data.items()}

In [ ]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

In [ ]:
# Let's take a look at the transcript for Ali Wong
data_df.transcript.loc['Nicolas Sarkozy 2010']

In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [ ]:
# Let's take a look at the updated text
data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

**NOTE:** This data cleaning aka text pre-processing step could go on for a while, but we are going to stop for now. After going through some analysis techniques, if you see that the results don't make sense or could be improved, you can come back and make more edits such as:
* Mark 'cheering' and 'cheer' as the same word (stemming / lemmatization)
* Combine 'thank you' into one term (bi-grams)
* And a lot more...

## Organizing The Data

I mentioned earlier that the output of this notebook will be clean, organized data in two standard text formats:
1. **Corpus - **a collection of text
2. **Document-Term Matrix - **word counts in matrix format

### Corpus

We already created a corpus in an earlier step. The definition of a corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.

In [ ]:
# Let's take a look at our dataframe
data_df

In [ ]:
# Let's add the comedians' full names as well
full_names = ['Ali Wong', 'Anthony Jeselnik', 'Bill Burr', 'Bo Burnham', 'Dave Chappelle', 'Hasan Minhaj',
              'Jim Jefferies', 'Joe Rogan', 'John Mulaney', 'Louis C.K.', 'Mike Birbiglia', 'Ricky Gervais']

#data_df['full_name'] = full_names
data_df['full_name'] = dates

data_df

In [ ]:
# Let's pickle it for later use
data_df.to_pickle("corpus_fr.pkl")

### Document-Term Matrix

For many of the techniques we'll be using in future notebooks, the text must be tokenized, meaning broken down into smaller pieces. The most common tokenization technique is to break down text into words. We can do this using scikit-learn's CountVectorizer, where every row will represent a different document and every column will represent a different word.

In addition, with CountVectorizer, we can remove stop words. Stop words are common words that add no additional meaning to text such as 'a', 'the', etc.

In [61]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

fsw = open('french_sw.txt', encoding='utf8').read().split('\n')
cv = CountVectorizer(stop_words=fsw)
data_cv = cv.fit_transform(data_clean.transcript)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,abaisse,abandon,abandonnent,abandonner,abandonné,abandonnée,abandonnés,abattues,abattus,abnégation,...,évolution,évolutions,évènementiel,évènements,événement,événements,êtees,êtes,êtreresponsables,êtres
Emmanuel Macron 2017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Emmanuel Macron 2018,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
Emmanuel Macron 2019,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
Emmanuel Macron 2020,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
François Hollande 2012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
François Hollande 2014,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
François Hollande 2015,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,1,0,0
François Hollande 2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
François Mitterrand 1981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
François Mitterrand 1982,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
# Let's pickle it for later use
data_dtm.to_pickle("dtm_fr.pkl")

In [60]:
# Let's also pickle the cleaned data (before we put it in document-term matrix format) and the CountVectorizer object
data_clean.to_pickle('data_clean_fr.pkl')
pickle.dump(cv, open("cv_fr.pkl", "wb"))


## Additional Exercises

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?